# 削除

まずはALBIngressControllerIAMPolicyの削除

In [129]:
ALBPolicyArn = 'arn:aws:iam::565991835156:policy/ALBIngressControllerIAMPolicy'

In [130]:
workerRolearn = 'eksctl-aws-eks-200123-nodegroup-n-NodeInstanceRole-1S46M9PWOP7U1'

In [131]:
!aws iam detach-role-policy \
--policy-arn {ALBPolicyArn} \
--role-name {workerRolearn}

In [132]:
!aws iam delete-policy \
--policy-arn {ALBPolicyArn}

kubectlで作成したオブジェクトの削除

In [133]:
!kubectl delete -f yaml/wordpress-ingress.yaml
!kubectl delete -f yaml/wordpress-service.yaml
!kubectl delete -f yaml/wordpress-deploy.yaml
!kubectl delete -f alb-ingress-controller.yaml
!kubectl delete -f https://raw.githubusercontent.com/kubernetes-sigs/aws-alb-ingress-controller/v1.1.4/docs/examples/rbac-role.yaml
!kubectl delete -f deployment.yaml    

ingress.extensions "wordpress-ingress" deleted
service "wordpress" deleted
deployment.apps "wordpress" deleted
deployment.apps "alb-ingress-controller" deleted
clusterrole.rbac.authorization.k8s.io "alb-ingress-controller" deleted
clusterrolebinding.rbac.authorization.k8s.io "alb-ingress-controller" deleted
serviceaccount "alb-ingress-controller" deleted
serviceaccount "efs-provisioner" deleted
deployment.apps "efs-provisioner" deleted


In [134]:
!kubectl delete -f class.yaml
!kubectl delete -f configmap.yaml
!kubectl delete -f rbac.yaml
!kubectl delete -f deployment.yaml
!kubectl delete -f claim.yaml

storageclass.storage.k8s.io "aws-efs" deleted
configmap "efs-provisioner" deleted
clusterrole.rbac.authorization.k8s.io "efs-provisioner-runner" deleted
clusterrolebinding.rbac.authorization.k8s.io "run-efs-provisioner" deleted
role.rbac.authorization.k8s.io "leader-locking-efs-provisioner" deleted
rolebinding.rbac.authorization.k8s.io "leader-locking-efs-provisioner" deleted
Error from server (NotFound): error when deleting "deployment.yaml": serviceaccounts "efs-provisioner" not found
Error from server (NotFound): error when deleting "deployment.yaml": deployments.apps "efs-provisioner" not found


RuntimeError: Unexpected exit code: 1

クラスタの削除

In [135]:
!eksctl delete cluster \
    --name {clusterName} \
    --wait

[ℹ]  eksctl version 0.13.0
[ℹ]  using region us-west-2
[ℹ]  deleting EKS cluster "aws-eks-200123"
[ℹ]  account is not authorized to use Fargate. Ignoring error
[ℹ]  cleaning up LoadBalancer services
[ℹ]  2 sequential tasks: { delete nodegroup "ng-200123", delete cluster control plane "aws-eks-200123" }
[ℹ]  will delete stack "eksctl-aws-eks-200123-nodegroup-ng-200123"
[ℹ]  waiting for stack "eksctl-aws-eks-200123-nodegroup-ng-200123" to get deleted
[✖]  unexpected status "DELETE_FAILED" while waiting for CloudFormation stack "eksctl-aws-eks-200123-nodegroup-ng-200123"
[ℹ]  fetching stack events in attempt to troubleshoot the root cause of the failure
[✖]  AWS::CloudFormation::Stack/eksctl-aws-eks-200123-nodegroup-ng-200123: DELETE_FAILED – "The following resource(s) failed to delete: [NodeInstanceRole]. "
[✖]  AWS::IAM::Role/NodeInstanceRole: DELETE_FAILED – "Cannot delete entity, must detach all policies first. (Service: AmazonIdentityManagement; Status Code: 409; Error Code: DeleteCo

RuntimeError: Unexpected exit code: 1